In [2]:
# Import dependencies
from pathlib import Path
import pandas as pd
from datetime import datetime
import plotly.graph_objects as go
import dash
import dash_daq as daq
from dash import dcc
from dash import html
import dash_html_components as html
from dash.dependencies import Input, Output


C:\Users\Yi\AppData\Local\Temp\ipykernel_40708\2400581193.py:10: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


# Cleaning data and creating Data Base

In [3]:
# Import csv file and create df
data_path = Path('Resources/Crime_Data_from_2020_to_Present.csv')
data_df = pd.read_csv(data_path)
data_df.head()

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
0,190326475,03/01/2020 12:00:00 AM,03/01/2020 12:00:00 AM,2130,7,Wilshire,784,1,510,VEHICLE - STOLEN,...,AA,Adult Arrest,510.0,998.0,NaN,NaN,1900 S LONGWOOD AV,NaN,34.0375,-118.3506
1,200106753,02/09/2020 12:00:00 AM,02/08/2020 12:00:00 AM,1800,1,Central,182,1,330,BURGLARY FROM VEHICLE,...,IC,Invest Cont,330.0,998.0,NaN,NaN,1000 S FLOWER ST,NaN,34.0444,-118.2628
2,200320258,11/11/2020 12:00:00 AM,11/04/2020 12:00:00 AM,1700,3,Southwest,356,1,480,BIKE - STOLEN,...,IC,Invest Cont,480.0,NaN,NaN,NaN,1400 W 37TH ST,NaN,34.0210,-118.3002
3,200907217,05/10/2023 12:00:00 AM,03/10/2020 12:00:00 AM,2037,9,Van Nuys,964,1,343,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),...,IC,Invest Cont,343.0,NaN,NaN,NaN,14000 RIVERSIDE DR,NaN,34.1576,-118.4387
4,220614831,08/18/2022 12:00:00 AM,08/17/2020 12:00:00 AM,1200,6,Hollywood,666,2,354,THEFT OF IDENTITY,...,IC,Invest Cont,354.0,NaN,NaN,NaN,1900 TRANSIENT,NaN,34.0944,-118.3277


In [4]:
# Keep columns in the dataframe
columns = ['DR_NO','DATE OCC', 'Vict Age']
data11_df = data_df[columns]
data11_df

,DR_NO,DATE OCC,Vict Age
0,190326475,03/01/2020 12:00:00 AM,0
1,200106753,02/08/2020 12:00:00 AM,47
2,200320258,11/04/2020 12:00:00 AM,19
3,200907217,03/10/2020 12:00:00 AM,19
4,220614831,08/17/2020 12:00:00 AM,28
...,...,...,...
944230,242106032,02/17/2024 12:00:00 AM,44
944231,242004546,01/16/2024 12:00:00 AM,80
944232,240104953,01/15/2024 12:00:00 AM,0
944233,240309674,04/24/2024 12:00:00 AM,70


In [5]:
# Obtain 'Year' column

# Split date and time from "DATE OCC" column
data11_df[['Date', 'Time']] = data11_df['DATE OCC'].str.split(' ', n=1, expand=True)

# Convert 'Date' to datetime using the correct format
data11_df['Date'] = pd.to_datetime(data11_df['Date'], format='%m/%d/%Y')

# Extract the year and create a new column 'Year'
data11_df['Year'] = data11_df['Date'].dt.year

# Drop not needed columns
data12_df = data11_df.drop(columns=['DATE OCC', 'Date', 'Time'])
data12_df.head()

C:\Users\Yi\AppData\Local\Temp\ipykernel_40708\2354198331.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data11_df[['Date', 'Time']] = data11_df['DATE OCC'].str.split(' ', n=1, expand=True)
C:\Users\Yi\AppData\Local\Temp\ipykernel_40708\2354198331.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data11_df[['Date', 'Time']] = data11_df['DATE OCC'].str.split(' ', n=1, expand=True)
C:\Users\Yi\AppData\Local\Temp\ipykernel_40708\2354198331.py:7: SettingWithCopyWarning: 
A value is trying to be set on a 

,DR_NO,Vict Age,Year
0,190326475,0,2020
1,200106753,47,2020
2,200320258,19,2020
3,200907217,19,2020
4,220614831,28,2020


In [6]:
# Rename the columns
data12_df = data12_df.rename(columns={'DR_NO': 'id', 'Vict Age': 'victim_age', 'Year':'year'})
data12_df.head()

,id,victim_age,year
0,190326475,0,2020
1,200106753,47,2020
2,200320258,19,2020
3,200907217,19,2020
4,220614831,28,2020


In [10]:
data12_df = data12_df.loc[data12_df['victim_age'] != 0]
data12_df.head()

,id,victim_age,year
1,200106753,47,2020
2,200320258,19,2020
3,200907217,19,2020
4,220614831,28,2020
5,231808869,41,2020


In [11]:
# Exclude data of 2024
data13_df = data12_df[data12_df['year'] != 2024]
data13_df

,id,victim_age,year
1,200106753,47,2020
2,200320258,19,2020
3,200907217,19,2020
4,220614831,28,2020
5,231808869,41,2020
...,...,...,...
876980,231606525,25,2023
876981,231716047,44,2023
876982,230115220,24,2023
876983,230906458,53,2023


# Visualizations and Dashboard

In [13]:
# Create the Dash app
app = dash.Dash(__name__)

# Define the app layout
app.layout = html.Div([
    html.H1('Los Angeles Crime Victim Age (2020-2023)', style={'textAlign': 'center'}),
    dcc.Graph(id='gauge-chart'),
    html.P('Filter by year:'),
    dcc.Dropdown(
        id='year-dropdown',
        options=[{'label': str(year), 'value': year} for year in data13_df['year'].unique()],
        value=data13_df['year'].min()
    ),
    html.P('Filter by age range:'),
    dcc.RangeSlider(
        id='age-range-slider',
        min=data13_df['victim_age'].min(),
        max=data13_df['victim_age'].max(),
        value=[data13_df['victim_age'].min(), data13_df['victim_age'].max()],
        marks={str(age): str(age) for age in range(int(data13_df['victim_age'].min()), int(data13_df['victim_age'].max()) + 1, 10)}
    )
])

# Define the callback function to update the gauge chart
@app.callback(
    Output('gauge-chart', 'figure'),
    [Input('year-dropdown', 'value'), Input('age-range-slider', 'value')])
def update_gauge_chart(selected_year, age_range):
    filtered_df = data13_df[(data13_df['year'] == selected_year) & (data13_df['victim_age'] >= age_range[0]) & (data13_df['victim_age'] <= age_range[1])]
    avg_age = filtered_df['victim_age'].mean()

    gauge_data = go.Indicator(
        domain={'x': [0, 1], 'y': [0, 1]},
        value=avg_age,
        mode="gauge+number",
        title={'text': "Average Victim Age"},
        gauge={
            'axis': {'range': [filtered_df['victim_age'].min(), filtered_df['victim_age'].max()]},
            'bar': {'color': "red"},
            'steps': [
                {'range': [0, avg_age], 'color': 'lightgrey'},
                {'range': [avg_age, filtered_df['victim_age'].max()], 'color': 'grey'}
            ]
        }
    )

    layout = go.Layout(
        title='Los Angeles Crime Victim Age (2020-2023)',
        font=dict(size=13)
    )

    return {'data': [gauge_data], 'layout': layout}

# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True)
